### Composio: Quick Start Tutorial

This notebook provides an up-to-date, step-by-step guide to interacting with **GitHub** using the **Composio** library. We'll follow the latest official documentation to build a simple AI agent that can perform actions on GitHub.

We will cover:
1.  **Setup**: Installing libraries and configuring API keys.
2.  **Authentication**: Securely connecting a user's GitHub account.
3.  **Fetching Tools**: Getting a list of available GitHub actions for our agent.
4.  **Executing Actions**: Using an LLM (like OpenAI's GPT) to understand a prompt and execute a GitHub action.

---
### 1. Setup

First, let's install the necessary libraries. We need `composio` for the tools and `openai` to act as the brain for our agent.

In [25]:
# Uncomment to install the packages
# !pip install composio==0.8.0 python-dotenv -q 

---
### 2. Authentication: Connecting to GitHub

Composio uses an `entity_id` to manage connections for each of your users. Here, we'll define a unique ID for our user and initiate the connection to GitHub. Composio handles the entire OAuth2 flow.

In [2]:
from dotenv import load_dotenv
from composio import Composio
import os

# Load environment variables from the .env file
load_dotenv()

# Initialize the Composio client
composio = Composio()

# Define a unique identifier for your user
user_id = os.environ["COMPOSIO_USER_ID"] 
# This can be a user ID from your database, an email, etc.

# Initiate a connection request for the user to authorize the 'github' toolkit
connection_request = composio.toolkits.authorize(user_id=user_id, toolkit="github")

print(f"🔗 Visit this URL to authorize GitHub:\n👉 {connection_request.redirect_url}")

# This will wait for the user to complete the authorization in their browser.
connection_request.wait_for_connection()
print("✅ GitHub has been successfully connected!")

🔗 Visit this URL to authorize GitHub:
👉 https://backend.composio.dev/api/v3/s/41iKBF5A
✅ GitHub has been successfully connected!


---
### 3. Fetching Available GitHub Tools

Now that the user is authenticated, we can fetch all the tools (actions) that Composio provides for the GitHub toolkit. These tools are formatted to be easily understood and used by an LLM.

In [6]:
# Fetch a list of all available tools for the GitHub toolkit for our user
github_tools = composio.tools.get(user_id=user_id, toolkits=["github"])

print(f"Found {len(github_tools)} tools for GitHub. Here are a few examples:")

# Print the name of the first 5 available tools
for tool in github_tools:
    print(f"- {tool['function']['name']}")

Found 20 tools for GitHub. Here are a few examples:
- GITHUB_ADD_ASSIGNEES_TO_AN_ISSUE
- GITHUB_ADD_A_REPOSITORY_COLLABORATOR
- GITHUB_ADD_A_SECURITY_MANAGER_TEAM
- GITHUB_ADD_EMAIL_FOR_AUTH_USER
- GITHUB_ADD_LABELS_TO_AN_ISSUE
- GITHUB_APPROVE_A_WORKFLOW_RUN_FOR_A_FORK_PULL_REQUEST
- GITHUB_COMPARE_TWO_COMMITS
- GITHUB_CREATE_AN_ISSUE
- GITHUB_CREATE_AN_ISSUE_COMMENT
- GITHUB_CREATE_AN_ORGANIZATION_INVITATION
- GITHUB_CREATE_AN_ORGANIZATION_REPOSITORY
- GITHUB_CREATE_A_BLOB
- GITHUB_CREATE_A_COMMIT
- GITHUB_CREATE_A_PULL_REQUEST
- GITHUB_CREATE_A_REFERENCE
- GITHUB_CREATE_A_RELEASE
- GITHUB_CREATE_A_TREE
- GITHUB_CREATE_OR_UPDATE_FILE_CONTENTS
- GITHUB_DELETE_A_REPOSITORY
- GITHUB_GET_A_PULL_REQUEST


---
### 4. Executing a GitHub Action with an LLM Agent

This is where the magic happens. We'll give a natural language prompt to an OpenAI model. We'll also provide it with the GitHub tools we just fetched. The model will understand the prompt, select the correct tool, and generate the necessary arguments.

Finally, `composio.provider.handle_tool_calls` will execute the action chosen by the LLM.

In [7]:
from openai import OpenAI

os.environ["OPENAI_API_KEY"] = os.environ["OPENAI_API_KEY"]

# Initialize the OpenAI client
client = OpenAI()

task_prompt = "Create a new issue in the repository devonsunml/ai_builders_tutorial with the title 'Improve documentation' and the body 'The documentation for the project could be improved by adding more examples and clarifying the setup instructions.'"

# Let the LLM choose the right tool for the task
print("🤖 Asking the LLM to choose a tool for the task...")
completion = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": task_prompt},
    ],
    tools=github_tools, # Provide the fetched tools to the model
    tool_choice="auto"
)

# Execute the tool call chosen by the LLM
print("⚡️ Executing the chosen tool...")
result = composio.provider.handle_tool_calls(user_id=user_id, response=completion)

print("\n✅ Task Completed!")
print("Final Response:", result)

🤖 Asking the LLM to choose a tool for the task...
⚡️ Executing the chosen tool...

✅ Task Completed!
Final Response: [{'data': {'active_lock_reason': None, 'assignee': None, 'assignees': [], 'author_association': 'OWNER', 'body': 'The documentation for the project could be improved by adding more examples and clarifying the setup instructions.', 'closed_at': None, 'closed_by': None, 'comments': 0, 'comments_url': 'https://api.github.com/repos/devonsunml/ai_builders_tutorial/issues/1/comments', 'created_at': '2025-08-05T16:58:31Z', 'events_url': 'https://api.github.com/repos/devonsunml/ai_builders_tutorial/issues/1/events', 'html_url': 'https://github.com/devonsunml/ai_builders_tutorial/issues/1', 'id': 3293794334, 'labels': [], 'labels_url': 'https://api.github.com/repos/devonsunml/ai_builders_tutorial/issues/1/labels{/name}', 'locked': False, 'milestone': None, 'node_id': 'I_kwDOPUmLMs7EU1Ae', 'number': 1, 'performed_via_github_app': None, 'reactions': {'+1': 0, '-1': 0, 'confused': 0

![Composio Github Demo](/composio.png)

### Summary
This notebook demonstrated how to use the Composio Python SDK to build an agent that can interact with GitHub. The workflow included:

- Authenticating a user and connecting their GitHub account via OAuth2.
- Fetching all available GitHub actions (tools) for the authenticated user.
- Using an LLM (OpenAI GPT) to select and execute a GitHub action based on a natural language prompt.
- Handling the tool call and displaying the result.

With this approach, you can easily extend your agent to support other tools and automate complex workflows using natural language and Composio's unified API.